In [6]:

from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

In [7]:
#https://cloud.google.com/docs/authentication/getting-started#command-line

credentials = service_account.Credentials.from_service_account_file(
    r"C:\Users\Nathan\AppData\Local\Google\Cloud SDK\service-acc-key.json")
project_id = 'bootcamp-bigquery-290701'

    

In [8]:

client = bigquery.Client(credentials = credentials,project = project_id)

In [9]:
query_job = client.query("""
SELECT
fullvisitorId
,visitId
,visitNumber
,visitStartTime
,date
,socialEngagementType
,channelGrouping
,device.browser
,device.operatingSystem
,device.deviceCategory
,geoNetwork.continent
,geoNetwork.subContinent
,geoNetwork.country
,geoNetwork.region
,geoNetwork.metro
,geoNetwork.city
,trafficSource.referralPath
,trafficSource.source
,trafficSource.medium
,trafficSource.isTrueDirect
,totals.bounces
,totals.hits
,totals.newVisits
,totals.pageviews
,totals.sessionQualityDim
,totals.totalTransactionRevenue
,totals.transactionRevenue
,totals.transactions
,totals.visits

FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
where _table_suffix between '20170501' and '20170831'

order by 1,2 nulls last
   LIMIT 200000 """)

results = query_job.result() # Wait for the job to complete.

In [10]:
totals_df = results.to_dataframe()

print(totals_df.shape)
totals_df.head()
# Having a lot of issues using pyarrow. I uninstalled the package and am getting things to work

C:\Users\Nathan\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """Entry point for launching an IPython kernel.


(200000, 29)


,fullvisitorId,visitId,visitNumber,visitStartTime,date,socialEngagementType,channelGrouping,browser,operatingSystem,deviceCategory,...,isTrueDirect,bounces,hits,newVisits,pageviews,sessionQualityDim,totalTransactionRevenue,transactionRevenue,transactions,visits
0,0000062267706107999,1499645960,1,1499645960,20170709,Not Socially Engaged,Affiliates,Chrome,Windows,desktop,...,None,1.0,1,1.0,1.0,1.0,NaN,NaN,NaN,1
1,0000085059828173212,1500505105,1,1500505105,20170719,Not Socially Engaged,Direct,Safari,iOS,mobile,...,True,1.0,1,1.0,1.0,1.0,NaN,NaN,NaN,1
2,0000197671390269035,1493772870,1,1493772870,20170502,Not Socially Engaged,Social,Android Webview,Android,mobile,...,None,1.0,1,1.0,1.0,NaN,NaN,NaN,NaN,1
3,000026722803385797,1496658394,1,1496658394,20170605,Not Socially Engaged,Organic Search,Safari,Macintosh,desktop,...,None,NaN,3,1.0,2.0,NaN,NaN,NaN,NaN,1
4,0000436683523507380,1500504900,1,1500504900,20170719,Not Socially Engaged,Referral,Chrome,Macintosh,desktop,...,None,NaN,2,1.0,2.0,1.0,NaN,NaN,NaN,1


In [11]:
totals_df = totals_df.fillna(0)
totals_df.isTrueDirect = totals_df.isTrueDirect.astype('bool')
totals_df.bounces = totals_df.bounces.astype('int')
totals_df.newVisits = totals_df.newVisits.astype('int')
totals_df.pageviews = totals_df.pageviews.astype('int')
totals_df.sessionQualityDim = totals_df.sessionQualityDim.astype('int')
totals_df.totalTransactionRevenue = totals_df.totalTransactionRevenue.astype('int64')
totals_df.transactionRevenue = totals_df.transactionRevenue.astype('int64')
totals_df.transactions = totals_df.transactions.astype('int')
totals_df.dtypes

fullvisitorId              object
visitId                     int64
visitNumber                 int64
visitStartTime              int64
date                       object
socialEngagementType       object
channelGrouping            object
browser                    object
operatingSystem            object
deviceCategory             object
continent                  object
subContinent               object
country                    object
region                     object
metro                      object
city                       object
referralPath               object
source                     object
medium                     object
isTrueDirect                 bool
bounces                     int32
hits                        int64
newVisits                   int32
pageviews                   int32
sessionQualityDim           int32
totalTransactionRevenue     int64
transactionRevenue          int64
transactions                int32
visits                      int64
dtype: object

In [ ]:
my_query = """
SELECT
distinct
fullVisitorID
,date

 FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
 WHERE _TABLE_SUFFIX BETWEEN '20170801' AND '20170805'
 ;
"""


query_job = client.query(my_query)

results = query_job.result() # Wait for the job to complete.

In [ ]:
customers_df = results.to_dataframe()
print(customers_df.shape)
customers_df.head()

# Connect to pgAdmin

In [1]:
from sqlalchemy import create_engine
from config import db_password2
import psycopg2

In [16]:
engine = psycopg2.connect(
    database = "postgres",
    user = "postgres",
    password = db_password2,
    host = "js-ucbfinalprojectdb.c8s02fywjegc.us-east-2.rds.amazonaws.com",
#     host = "dataviz-db.csxrf9ti2aba.us-east-2.rds.amazonaws.com",
    port = 5432)
cursor = engine.cursor()

In [17]:
#use Pandas to see all data in table
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=engine)

,table_name,column_name,data_type,table_schema
0,bigquery_totals,transactions,integer,public
1,bigquery_totals,browser,character varying,public
2,bigquery_totals,visits,integer,public
3,bigquery_totals,socialengagementtype,character varying,public
4,bigquery_totals,visitstarttime,integer,public
...,...,...,...,...
94,sessions,visitid,integer,public
95,sessions,fullvisitorid,character varying,public
96,sessions_test,date,date,public
97,sessions_test,fullvisitorid,character varying,public


# Load customers and then insert them

In [ ]:
sql = """drop table customers_test2;"""

cursor.execute(sql)
sql = """
CREATE TABLE customers_test2 (
     fullvisitorId VARCHAR(19) NOT NULL,
	 date DATE NOT NULL
	 --PRIMARY KEY (fullvisitorId),
);
"""
cursor.execute(sql)
engine.commit()

In [14]:
from io import StringIO

#use stringIO to initizalize a string buffer so we can "copy" the csv from the buffer into the dB

# Initialize a string buffer
sio = StringIO()
sio.write(customers_df.to_csv(index = None,header = None)) #Write the Pandas df as a csv to the buffer
sio.seek(0) # be sure to reset the position to the start of the stream

# Copy the string buffer into the db as if it were an actual file
with engine.cursor() as c:
    c.copy_from (sio,"customers_test2",columns = customers_df.columns,sep = ',')
    engine.commit()

NameError: name 'customers_df' is not defined

In [12]:
sql = """delete from bigquery_totals;"""
cursor.execute(sql)
engine.commit()


In [18]:
# Initialize a string buffer
sio = StringIO()
sio.write(totals_df.to_csv(index = None,header = None,sep = "|")) #Write the Pandas df as a csv to the buffer
sio.seek(0) # be sure to reset the position to the start of the stream

# Copy the string buffer into the db as if it were an actual file
with engine.cursor() as c:
    c.copy_from (sio,"bigquery_totals",columns = totals_df.columns,sep = '|')
    engine.commit()

StringDataRightTruncation: value too long for type character varying(255)
CONTEXT:  COPY bigquery_totals, line 90531, column referralpath: "/web/search/google+/@37.418619,-122.087756,10.27337463a,818.4229692d,35y,0h,45t,0r/data=CoMBGlkSUwol..."


In [ ]:
engine.commit()